In [ ]:
import math as m
import numpy as np
import random
from scipy.spatial import cKDTree
import time
import matplotlib.pyplot as plt
from multiprocessing.dummy import Pool
from scipy.optimize import curve_fit

# I. Modèle Viscek basique

On implémente les constantes du programme (nombre d'itérations, rayon d'interaction,...)

In [ ]:
#%% Définition des variables globales

r=1 #interaction radius
delta_t=1 #time interval
T=100 #nombre d'itération


On commence par implémenter les classes nécessaires aux calculs, i.e une nuée et son évolution.

In [ ]:
class Flock:
    def __init__(self, eta, rho, L, v):
        self.eta = eta
        self.rho = rho
        self.L = L
        self.v = v
        self.N = int(rho * L**2)
        
        # Initialisation
        self.position = np.random.rand(self.N, 2) * L
        self.angle = np.random.rand(self.N) * 2 * np.pi
        self.vitesse = np.zeros((self.N, 2))
        self.vitesse[:, 0] = v * np.cos(self.angle)
        self.vitesse[:, 1] = v * np.sin(self.angle)
    
    def evolve(self):
        delta_teta = np.random.uniform(-self.eta/2, self.eta/2, size=self.N)
        tree = cKDTree(self.position)
        neighbors_list = tree.query_ball_point(self.position, r)
        
        new_angle = np.zeros(self.N)
        new_position = np.zeros((self.N, 2))
        new_vitesse = np.zeros((self.N, 2))
        
        for i in range(self.N):
            voisins = [j for j in neighbors_list[i] if i != j]
            
            if voisins:
                somme_sin = np.sum(np.sin(self.angle[voisins]))
                somme_cos = np.sum(np.cos(self.angle[voisins]))
            else:
                somme_sin = 0
                somme_cos = 0
            
            moyenne_angle = np.arctan2(somme_sin, somme_cos)
            new_angle[i] = moyenne_angle + delta_teta[i]
            new_position[i, 0] = (self.position[i, 0] + self.v * np.cos(new_angle[i]) * delta_t) % self.L
            new_position[i, 1] = (self.position[i, 1] + self.v * np.sin(new_angle[i]) * delta_t) % self.L
            new_vitesse[i, 0] = self.v * np.cos(new_angle[i])
            new_vitesse[i, 1] = self.v * np.sin(new_angle[i])
        
        self.position = new_position
        self.angle = new_angle
        self.vitesse = new_vitesse

    def polarisation(self):
        phi = np.sqrt((np.sum(self.vitesse[:,0]))**2 + (np.sum(self.vitesse[:,1]))**2) / (self.N*self.v)
        return phi

On ajoute les fonctions d'execution du modèle

In [ ]:
# Execution

def exe(eta, rho, L, v):
    flock = Flock(eta, rho, L, v)
    for t in range(T):
        flock.evolve()
    return flock.polarisation()

# L'utilisation de multiprocessing pour paralléliser les simulations a été ajoutée
# grâce à chatGPT pour améliorer les performances lors de l'exécution.
def worker(args):
    eta, rho, L, v = args
    pol = exe(eta, rho, L, v)
    return eta, pol

On affiche la disposition des particules dans la grille après les T pas de temps, pour différentes conditions

In [ ]:
# Création d'une instance de Flock
# Paramètres globaux
N = 300      # nombre de particules
v = 0.03     # vitesse


# Définition des trois nuées avec N fixé mais L variable
flocks = [
    {"label": "Peu dense, peu de bruit", "L": 25, "eta": 0.1},
    {"label": "Dense, beaucoup de bruit", "L": 7, "eta": 2.0},
    {"label": "Dense, peu de bruit", "L": 5, "eta": 0.1}
]

for flock_params in flocks:
    # Calcul de la densité
    rho = N / flock_params["L"]**2
    flock = Flock(eta=flock_params["eta"], rho=rho, L=flock_params["L"], v=v)

    # Affichage initial
    plt.figure(figsize=(6,6))
    plt.quiver(flock.position[:,0], flock.position[:,1],
               flock.vitesse[:,0], flock.vitesse[:,1],
               angles='xy', scale_units='xy', scale=0.05, color='blue', width=0.005)
    plt.xlim(0, flock_params["L"])
    plt.ylim(0, flock_params["L"])
    plt.title(f"{flock_params['label']} - Positions initiales")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.show()

    # Evolution du système
    for t in range(T):
        flock.evolve()

    # Affichage final
    plt.figure(figsize=(6,6))
    plt.quiver(flock.position[:,0], flock.position[:,1],
               flock.vitesse[:,0], flock.vitesse[:,1],
               angles='xy', scale_units='xy', scale=0.05, color='red', width=0.005)
    plt.xlim(0, flock_params["L"])
    plt.ylim(0, flock_params["L"])
    plt.title(f"{flock_params['label']} - Positions finales après {T} itérations")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.show()


On calcule la variation du paramètre d'ordre $\Phi$ en fonction de $\eta$ pour $\rho=4$

In [ ]:
def analyse(rho, L, v, n_iterations):
    params = [(0.25 * i, rho, L, v) for i in range(20) for _ in range(n_iterations)]
    
    # Utilisation de multiprocessing pour paralléliser les simulations
    with Pool(4) as pool:
        results = pool.map(worker, params)
    
    # Regrouper par eta
    eta_dict = {}
    for eta, pol in results:
        if eta not in eta_dict:
            eta_dict[eta] = []
        eta_dict[eta].append(pol)
    
    # Calculer moyenne et écart-type
    Etas = sorted(eta_dict.keys())
    Pol_mean = [np.mean(eta_dict[eta]) for eta in Etas]
    Pol_std = [np.std(eta_dict[eta]) for eta in Etas]
    
    return list(Etas), list(Pol_mean), list(Pol_std)

In [ ]:
rho = 4
L = 5
v = 0.03
Etas, Pol, Pol_std = analyse(rho, L, v, 5)
plt.figure(figsize=(8, 8))
plt.errorbar(Etas, Pol, yerr=Pol_std, fmt='+')
plt.xlabel(r'$\eta$')
plt.ylabel(r'$\Phi$')
plt.title('Phi en fonction de eta (rho constant)')
plt.show()

On réalise le diagramme des phases

In [ ]:
# Trouver le ppoint d'inflexion où la polarisation passe de ~1 à ~0
def find_inflection_point(etas, polarizations, pol_std):
    # Calculer les dérivées numériques de la polarisation par rapport à eta
    derivatives = np.diff(polarizations) / np.diff(etas)
    # Trouver l'indice où la dérivée est minimale (transition la plus abrupte)
    inflection_idx = np.argmin(derivatives)
    # Convertir l'écart type de la polarisation en écart type sur eta
    eta_std = pol_std[inflection_idx] / abs(derivatives[inflection_idx])
    # Retourner la valeur de eta, de polarisation et l'écart type sur eta
    return etas[inflection_idx], eta_std

def diagramme_phase(L,v, n_iterations):
    Rhos = [0.2*rho for rho in range(50)]
    Eta_crit = []
    Eta_crit_error = []
    for rho in Rhos:
        Etas, Pol_mean, Pol_std = analyse(rho, L, v, n_iterations)
        eta_crit, eta_crit_std = find_inflection_point(Etas, Pol_mean, Pol_std)
        Eta_crit.append(eta_crit)
        Eta_crit_error.append(eta_crit_std)
    return Rhos, Eta_crit, Eta_crit_error

On l'affiche. Attention, l'execution est très lente (~20min)

In [ ]:

L = 5
v = 0.03
Rhos, Eta_crit, Eta_crit_error = diagramme_phase(L,v, 10)


# Conversion en arrays
Rhos = np.array(Rhos)
Eta_crit = np.array(Eta_crit)
Eta_crit_error = np.array(Eta_crit_error)

# Remplacer erreurs nulles par un petit nombre
Eta_crit_error = np.where(Eta_crit_error == 0, 1e-6, Eta_crit_error)

# Exclure rho=0 pour le fitting si nécessaire
mask = Rhos > 0
Rhos_fit_data = Rhos[mask]
Eta_crit_fit_data = Eta_crit[mask]
Eta_crit_error_fit_data = Eta_crit_error[mask]

# Modèle exponentiel saturant
def exp_model(rho, alpha, beta):
    return alpha * (1 - np.exp(-beta * rho))

# Ajustement pondéré avec bornes (alpha>0, beta>0)
params, cov = curve_fit(
    exp_model,
    Rhos_fit_data,
    Eta_crit_fit_data,
    bounds=([0,0],[np.inf,np.inf])
)
alpha, beta = params

# Courbe lisse
Rhos_fit = np.linspace(0, max(Rhos), 300)
Eta_fit = exp_model(Rhos_fit, alpha, beta)

# Incertitude via propagation d'erreur (avec terme croisé)
dY_dalpha = 1 - np.exp(-beta * Rhos_fit)
dY_dbeta = alpha * Rhos_fit * np.exp(-beta * Rhos_fit)
Eta_fit_err = np.sqrt(
    dY_dalpha**2 * cov[0,0] + dY_dbeta**2 * cov[1,1] + 2*dY_dalpha*dY_dbeta*cov[0,1]
)

# Tracé
plt.figure(figsize=(8, 8))
plt.fill_between(Rhos, Eta_crit, max(Eta_crit)*1.5, color='red', alpha=0.3, label='état désordonné')
plt.fill_between(Rhos, 0, Eta_crit, color='blue', alpha=0.3, label='état ordonné')
plt.errorbar(Rhos, Eta_crit, yerr=Eta_crit_error, fmt='o', capsize=4, label='données')

plt.plot(Rhos_fit, Eta_fit, 'k--', linewidth=2, label='tendance exponentielle')
plt.fill_between(Rhos_fit, Eta_fit - Eta_fit_err, Eta_fit + Eta_fit_err, color='gray', alpha=0.3, label='incertitude ajustement')

# Ajouter l'équation sur le graphe
equation_text = f"$\\eta_c(\\rho) = {alpha:.3f} \\cdot (1 - e^{{-{beta:.3f} \\rho}})$"
plt.text(0.05*max(Rhos), 0.9*max(Eta_crit), equation_text, fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.5))

plt.legend()
plt.xlabel(r"$\rho$")
plt.ylabel(r"$\eta_c$")
plt.title('Diagramme des phases du modèle de Viscek pour L = 5')
plt.show()

On trace $log(\Phi)$ en fonction de $log(\frac{\eta_c - \eta}{\eta_c})$ pour déterminer l'exposant critique

In [ ]:
def analyse_transition(rho, L, v, n_iterations):
    params = [(0.07 * i, rho, L, v) for i in range(50) for _ in range(n_iterations)]
    #on doit s'arrêter à eta < eta_c
    with Pool(4) as pool:
        results = pool.map(worker, params)
    
    # Regrouper par eta
    eta_dict = {}
    for eta, pol in results:
        if eta not in eta_dict:
            eta_dict[eta] = []
        eta_dict[eta].append(pol)
    
    # Calculer moyenne et écart-type
    Etas = sorted(eta_dict.keys())
    Pol_mean = [np.mean(eta_dict[eta]) for eta in Etas]
    Pol_std = [np.std(eta_dict[eta]) for eta in Etas]

    return list(Etas), list(Pol_mean), list(Pol_std)

In [ ]:
# Paramètres
L = 5
v = 0.03
rho = 5
eta_c = 3.5

# Analyse
Etas, Pol, Pol_std = analyse_transition(rho, L, v,10)

# Conversion en tableaux NumPy
Etas = np.array(Etas)
Pol = np.array(Pol)
Pol_std = np.array(Pol_std)

# Variables log-log
x = (eta_c - Etas) / eta_c
y = Pol

# Filtrage valeurs valides
mask = (x > 0) & (y > 0)
x = x[mask]
y = y[mask]

# Fit log-log
logx = np.log(x)
logy = np.log(y)

slope, intercept = np.polyfit(logx, logy, 1)

# Courbe de fit
x_fit = np.linspace(x.min(), x.max(), 200)
y_fit = np.exp(intercept) * x_fit**slope

# Tracé
plt.figure()
plt.loglog(x, y, 'o', label='Données')
plt.loglog(x_fit, y_fit, '-', 
        label=rf'Fit : $P \sim x^{{{slope:.2f}}}$')

plt.ylim(bottom=0.1)
plt.xlabel(r'$(\eta_c - \eta)/\eta_c$')
plt.ylabel(r'$\mathrm{Pol}$')
plt.grid(True, which='both')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Exposant (pente log-log) = {slope:.4f}")

# II. Modification du bruit

## 1. Bruit gaussien

On commence par modifier la classe flock pour inclure un bruit gaussien

In [ ]:
from scipy.stats import truncnorm


def evolve_gaussien(self):
    delta_teta = truncnorm.rvs(-np.pi, np.pi, loc=0, scale=self.eta/2, size=self.N)
    tree = cKDTree(self.position)
    neighbors_list = tree.query_ball_point(self.position, r)
    
    new_angle = np.zeros(self.N)
    new_position = np.zeros((self.N, 2))
    new_vitesse = np.zeros((self.N, 2))
    
    for i in range(self.N):
        voisins = [j for j in neighbors_list[i] if i != j]
        
        if voisins:
            somme_sin = np.sum(np.sin(self.angle[voisins]))
            somme_cos = np.sum(np.cos(self.angle[voisins]))
        else:
            somme_sin = 0
            somme_cos = 0
        
        moyenne_angle = np.arctan2(somme_sin, somme_cos)
        new_angle[i] = moyenne_angle + delta_teta[i]
        new_position[i, 0] = (self.position[i, 0] + self.v * np.cos(new_angle[i]) * delta_t) % self.L
        new_position[i, 1] = (self.position[i, 1] + self.v * np.sin(new_angle[i]) * delta_t) % self.L
        new_vitesse[i, 0] = self.v * np.cos(new_angle[i])
        new_vitesse[i, 1] = self.v * np.sin(new_angle[i])
    
    self.position = new_position
    self.angle = new_angle
    self.vitesse = new_vitesse

Flock.evolve_gaussien = evolve_gaussien

# Execution

def exe_gaussien(eta, rho, L, v):
    flock = Flock(eta, rho, L, v)
    for t in range(T):
        flock.evolve_gaussien()
    return flock.polarisation()

def worker_gaussien(args):
    eta, rho, L, v = args
    pol = exe_gaussien(eta, rho, L, v)
    return eta, pol

def analyse_gaussien(rho, L, v, n_iterations):
    params = [(0.25 * i, rho, L, v) for i in range(0,20) for _ in range(n_iterations)]
    
    with Pool(4) as pool:
        results = pool.map(worker_gaussien, params)
    
    # Regrouper par eta
    eta_dict = {}
    for eta, pol in results:
        if eta not in eta_dict:
            eta_dict[eta] = []
        eta_dict[eta].append(pol)
    
    # Calculer moyenne et écart-type
    Etas = sorted(eta_dict.keys())
    Pol_mean = [np.mean(eta_dict[eta]) for eta in Etas]
    Pol_std = [np.std(eta_dict[eta]) for eta in Etas]
    
    return list(Etas), list(Pol_mean), list(Pol_std)

Voyons l'évolution de $\Phi$ en fonction de $\eta$, comme dans la première partie afin d'évaluer l'influence de cette nouvelle distribution de bruit.

In [ ]:
rho = 4
L = 5
v = 0.03
Etas, Pol, Pol_std = analyse_gaussien(rho, L, v, 10)
plt.figure(figsize=(8, 8))
plt.errorbar(Etas, Pol, yerr=Pol_std, fmt='+')

plt.xlabel(r'$\eta$')
plt.ylabel(r'$\Phi$')
plt.ylim(bottom=0)
plt.title('Phi en fonction de eta (rho constant)')
plt.show()

## 2. Bruit extrinsèque

In [ ]:
def evolve_ext(self):
    # construction de l'arbre pour voisins
    tree = cKDTree(self.position)
    neighbors_list = tree.query_ball_point(self.position, r)

    new_angle = np.zeros(self.N)
    new_position = np.zeros((self.N, 2))
    new_vitesse = np.zeros((self.N, 2))

    for i in range(self.N):
        voisins = [j for j in neighbors_list[i] if j != i]

        if voisins:
            # bruit extrinsèque unique pour ce champ moyen
            delta_theta = truncnorm.rvs(-np.pi, np.pi, loc=0, scale=self.eta/2)

            # appliquer le bruit aux angles des voisins avant de calculer la moyenne
            somme_sin = np.sum(np.sin(self.angle[voisins] + delta_theta))
            somme_cos = np.sum(np.cos(self.angle[voisins] + delta_theta))
            new_angle[i] = np.arctan2(somme_sin, somme_cos)
        else:
            new_angle[i] = self.angle[i]

        # mise à jour position
        new_position[i, 0] = (self.position[i, 0] + self.v * np.cos(new_angle[i]) * delta_t) % self.L
        new_position[i, 1] = (self.position[i, 1] + self.v * np.sin(new_angle[i]) * delta_t) % self.L

        # mise à jour vitesse
        new_vitesse[i, 0] = self.v * np.cos(new_angle[i])
        new_vitesse[i, 1] = self.v * np.sin(new_angle[i])

    self.position = new_position
    self.angle = new_angle
    self.vitesse = new_vitesse

Flock.evolve_ext = evolve_ext

# Execution
def exe_ext(eta, rho, L, v):
    flock = Flock(eta, rho, L, v)
    for t in range(T):
        flock.evolve_ext()
    return flock.polarisation()

def worker_ext(args):
    eta, rho, L, v = args
    pol = exe_ext(eta, rho, L, v)
    return eta, pol

def analyse_ext(rho, L, v, n_iterations):
    params = [(0.25 * i, rho, L, v) for i in range(0,20) for _ in range(n_iterations)]
    
    with Pool(4) as pool:
        results = pool.map(worker_ext, params)
    
    # Regrouper par eta
    eta_dict = {}
    for eta, pol in results:
        if eta not in eta_dict:
            eta_dict[eta] = []
        eta_dict[eta].append(pol)
    
    # Calculer moyenne et écart-type
    Etas = sorted(eta_dict.keys())
    Pol_mean = [np.mean(eta_dict[eta]) for eta in Etas]
    Pol_std = [np.std(eta_dict[eta]) for eta in Etas]
    
    return list(Etas), list(Pol_mean), list(Pol_std)

Voyons la répartition des particules pour un bruit extrinsèque

In [ ]:

rho = 4
L = 5
v = 0.03
Etas, Pol, Pol_std = analyse_ext(rho, L, v, 10)
plt.figure(figsize=(8, 8))
plt.errorbar(Etas, Pol, yerr=Pol_std, fmt='+')

plt.xlabel(r'$\eta$')
plt.ylabel(r'$\Phi$')
plt.ylim(bottom=0)
plt.title('Phi en fonction de eta (rho constant)')
plt.show()

Etudions l'évolution de la corrélation spatiale

In [ ]:
#    Calcule la corrélation spatiale des vitesses : C(r)
#    r_max : distance maximale pour calculer la corrélation
#    dr : largeur des bins
def spatial_correlation(self, r_max, dr):
   
    N = self.N
    r_bins = np.arange(0, r_max + dr, dr)
    C_r = np.zeros(len(r_bins)-1)
    counts = np.zeros(len(r_bins)-1)
    
    tree = cKDTree(self.position, boxsize=self.L)  # conditions périodiques
    
    for i in range(N):
        neighbors = tree.query_ball_point(self.position[i], r_max)
        for j in neighbors:
            if j == i:
                continue
            # vecteur distance minimal avec conditions périodiques
            delta = self.position[j] - self.position[i]
            delta = delta - self.L * np.round(delta / self.L)
            dist = np.linalg.norm(delta)
            
            # produit scalaire normalisé
            dot = np.dot(self.vitesse[i], self.vitesse[j]) / (self.v**2)
            
            # trouver le bin
            bin_index = int(dist / dr)
            if bin_index < len(C_r):
                C_r[bin_index] += dot
                counts[bin_index] += 1
    
    C_r = C_r / np.maximum(counts, 1)
    r_bin_centers = r_bins[:-1] + dr/2
    
    return r_bin_centers, C_r

Flock.spatial_correlation = spatial_correlation

# Affichage de la corrélation spatiale pour une nuée donnée
flock_int = Flock(eta=2.5, rho=5, L=20, v=0.03)
flock_ext = Flock(eta=2.5, rho=5, L=20, v=0.03)
# faire évoluer la simulation
for t in range(T):
    flock_ext.evolve_ext()
    flock_int.evolve_gaussien()

# calculer la corrélation spatiale
r_int, C_r_int = flock_int.spatial_correlation(r_max=10, dr=0.1)
r_ext, C_r_ext = flock_ext.spatial_correlation(r_max=10, dr=0.1)


plt.plot(r_int, C_r_int, label='Bruit intrinsèque')
plt.xlabel("Distance r")
plt.ylabel("C(r)")
plt.ylim(bottom=0)
plt.title("Corrélation spatiale des vitesses")
plt.show()
plt.plot(r_ext, C_r_ext, label='Bruit extrinsèque', color='orange')
plt.xlabel("Distance r")
plt.ylabel("C(r)")
plt.ylim(bottom=0)
plt.title("Corrélation spatiale des vitesses")
plt.show()

Superposons les deux courbes

In [ ]:
plt.plot(r_ext, C_r_ext, label='Bruit extrinsèque', color='orange')
plt.plot(r_int, C_r_int, label='Bruit intrinsèque')
plt.legend()
plt.xlabel("Distance r")
plt.ylabel("C(r)")
plt.ylim(bottom=0)
plt.title("Corrélation spatiale des vitesses")
plt.show()

## 3. Bruit perceptif



In [ ]:
# Evolution avec bruit gaussien dépendant de la distance
def evolve_distance(self):

    tree = cKDTree(self.position)
    neighbors_list = tree.query_ball_point(self.position, r)
    
    new_angle = np.zeros(self.N)
    new_position = np.zeros((self.N, 2))
    new_vitesse = np.zeros((self.N, 2))
    
    for i in range(self.N):
        voisins = [j for j in neighbors_list[i] if j != i]
        
        if voisins:
            # calcul des vecteurs distances avec conditions périodiques
            delta = self.position[voisins] - self.position[i]
            delta = delta - self.L * np.round(delta / self.L)
            dist = np.linalg.norm(delta, axis=1)
            
            # bruit gaussien proportionnel à la distance
            delta_theta = truncnorm.rvs(-np.pi, np.pi, loc=0, scale=self.eta * dist/ 2)
            
            # angles bruités des voisins
            angles_bruites = self.angle[voisins] + delta_theta
            
            # moyenne des angles bruités
            somme_sin = np.sum(np.sin(angles_bruites))
            somme_cos = np.sum(np.cos(angles_bruites))
            new_angle[i] = np.arctan2(somme_sin, somme_cos)
        else:
            new_angle[i] = self.angle[i]
        
        # mise à jour position
        new_position[i, 0] = (self.position[i, 0] + self.v * np.cos(new_angle[i]) * delta_t) % self.L
        new_position[i, 1] = (self.position[i, 1] + self.v * np.sin(new_angle[i]) * delta_t) % self.L
        
        # mise à jour vitesse
        new_vitesse[i, 0] = self.v * np.cos(new_angle[i])
        new_vitesse[i, 1] = self.v * np.sin(new_angle[i])
    
    self.position = new_position
    self.angle = new_angle
    self.vitesse = new_vitesse

Flock.evolve_distance = evolve_distance

# Execution
def exe_distance(eta, rho, L, v):
    flock = Flock(eta, rho, L, v)
    for t in range(T):
        flock.evolve_distance()
    return flock.polarisation()

def worker_distance(args):
    eta, rho, L, v = args
    pol = exe_distance(eta, rho, L, v)
    return eta, pol

def analyse_distance(rho, L, v, n_iterations):
    params = [(0.25 * i, rho, L, v) for i in range(0,20) for _ in range(n_iterations)]
    
    with Pool(4) as pool:
        results = pool.map(worker_distance, params)
    
    # Regrouper par eta
    eta_dict = {}
    for eta, pol in results:
        if eta not in eta_dict:
            eta_dict[eta] = []
        eta_dict[eta].append(pol)
    
    # Calculer moyenne et écart-type
    Etas = sorted(eta_dict.keys())
    Pol_mean = [np.mean(eta_dict[eta]) for eta in Etas]
    Pol_std = [np.std(eta_dict[eta]) for eta in Etas]
    
    return list(Etas), list(Pol_mean), list(Pol_std)

On affiche la corrélation spatiale du modèle de bruit proportionnel à la distance

In [ ]:
# Affichage de la corrélation spatiale pour une nuée donnée
flock_int = Flock(eta=2.5, rho=5, L=20, v=0.03)
flock_ext = Flock(eta=2.5, rho=5, L=20, v=0.03)
# faire évoluer la simulation
for t in range(T):
    flock_ext.evolve_distance()
    flock_int.evolve_gaussien()

# calculer la corrélation spatiale
r_int, C_r_int = flock_int.spatial_correlation(r_max=10, dr=0.1)
r_ext, C_r_ext = flock_ext.spatial_correlation(r_max=10, dr=0.1)


plt.plot(r_int, C_r_int, label='Bruit intrinsèque')
plt.xlabel("Distance r")
plt.ylabel("C(r)")
plt.ylim(bottom=0)
plt.title("Corrélation spatiale des vitesses")
plt.show()
plt.plot(r_ext, C_r_ext, label='Bruit extrinsèque', color='orange')
plt.xlabel("Distance r")
plt.ylabel("C(r)")
plt.ylim(bottom=0)
plt.title("Corrélation spatiale des vitesses")
plt.show()

## III. Comparaison modèle raffiné / modèle basique

On affiche la répartition des particules en fin de simulation (analyse qualitative)

In [ ]:
# Création d'une instance de Flock
# Paramètres globaux
N = 300      # nombre de particules
v = 0.03     # vitesse


# Définition des trois nuées avec N fixé mais L variable
flocks = [
    {"label": "Peu dense, peu de bruit", "L": 25, "eta": 0.1},
    {"label": "Dense, beaucoup de bruit", "L": 7, "eta": 2.0},
    {"label": "Dense, peu de bruit", "L": 5, "eta": 0.1}
]

for flock_params in flocks:
    # Calcul de la densité
    rho = N / flock_params["L"]**2
    flock = Flock(eta=flock_params["eta"], rho=rho, L=flock_params["L"], v=v)


    # Evolution du système
    for t in range(T):
        flock.evolve_distance()

    # Affichage final
    plt.figure(figsize=(6,6))
    plt.quiver(flock.position[:,0], flock.position[:,1],
               flock.vitesse[:,0], flock.vitesse[:,1],
               angles='xy', scale_units='xy', scale=0.05, color='red', width=0.005)
    plt.xlim(0, flock_params["L"])
    plt.ylim(0, flock_params["L"])
    plt.title(f"{flock_params['label']} - Positions finales après {T} itérations")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.show()


On trace le diagramme des phases pour déduire l'exposant critique et voir la "tolérance au bruit" du nouveau modèle

In [ ]:
# Calcul

def diagramme_phase_distance(L,v, n_iterations):
    Rhos = [0.2*rho for rho in range(50)]
    Eta_crit = []
    Eta_crit_error = []
    for rho in Rhos:
        Etas, Pol_mean, Pol_std = analyse_distance(rho, L, v, n_iterations)
        eta_crit, eta_crit_std = find_inflection_point(Etas, Pol_mean, Pol_std)
        Eta_crit.append(eta_crit)
        Eta_crit_error.append(eta_crit_std)
    return Rhos, Eta_crit, Eta_crit_error

Attention, l'execution est très lente (~20min)

In [ ]:
#Affichage du diagramme de phase pour le modèle avec bruit dépendant de la distance

L = 5
v = 0.03
Rhos, Eta_crit, Eta_crit_error = diagramme_phase_distance(L,v, 10)


# Conversion en arrays
Rhos = np.array(Rhos)
Eta_crit = np.array(Eta_crit)
Eta_crit_error = np.array(Eta_crit_error)

# Remplacer erreurs nulles par un petit nombre
Eta_crit_error = np.where(Eta_crit_error == 0, 1e-6, Eta_crit_error)

# Exclure rho=0 pour le fitting si nécessaire
mask = Rhos > 0
Rhos_fit_data = Rhos[mask]
Eta_crit_fit_data = Eta_crit[mask]
Eta_crit_error_fit_data = Eta_crit_error[mask]

# Modèle exponentiel saturant
def exp_model(rho, alpha, beta):
    return alpha * (1 - np.exp(-beta * rho))

# Ajustement pondéré avec bornes (alpha>0, beta>0)
params, cov = curve_fit(
    exp_model,
    Rhos_fit_data,
    Eta_crit_fit_data,
    bounds=([0,0],[np.inf,np.inf])
)
alpha, beta = params

# Courbe lisse
Rhos_fit = np.linspace(0, max(Rhos), 300)
Eta_fit = exp_model(Rhos_fit, alpha, beta)

# Incertitude via propagation d'erreur (avec terme croisé)
dY_dalpha = 1 - np.exp(-beta * Rhos_fit)
dY_dbeta = alpha * Rhos_fit * np.exp(-beta * Rhos_fit)
Eta_fit_err = np.sqrt(
    dY_dalpha**2 * cov[0,0] + dY_dbeta**2 * cov[1,1] + 2*dY_dalpha*dY_dbeta*cov[0,1]
)

# Tracé
plt.figure(figsize=(8, 8))
plt.fill_between(Rhos, Eta_crit, max(Eta_crit)*1.5, color='red', alpha=0.3, label='état désordonné')
plt.fill_between(Rhos, 0, Eta_crit, color='blue', alpha=0.3, label='état ordonné')
plt.errorbar(Rhos, Eta_crit, yerr=Eta_crit_error, fmt='o', capsize=4, label='données')

plt.plot(Rhos_fit, Eta_fit, 'k--', linewidth=2, label='tendance exponentielle')
plt.fill_between(Rhos_fit, Eta_fit - Eta_fit_err, Eta_fit + Eta_fit_err, color='gray', alpha=0.3, label='incertitude ajustement')

# Ajouter l'équation sur le graphe
equation_text = f"$\\eta_c(\\rho) = {alpha:.3f} \\cdot (1 - e^{{-{beta:.3f} \\rho}})$"
plt.text(0.05*max(Rhos), 0.9*max(Eta_crit), equation_text, fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.5))

plt.legend()
plt.xlabel(r"$\rho$")
plt.ylabel(r"$\eta_c$")
plt.title('Diagramme des phases du modèle de Viscek raffiné pour L = 5')
plt.show()

Enfin, on trace $log(\Phi)$ en fonction de $log(\frac{\eta_c - \eta}{\eta_c})$ pour déterminer l'exposant critique

In [ ]:
def analyse_transition_distance(rho, L, v, n_iterations):
    params = [(0.2 * i, rho, L, v) for i in range(20) for _ in range(n_iterations)]
    #on doit s'arrêter à eta < eta_c
    with Pool(4) as pool:
        results = pool.map(worker_distance, params)
    
    # Regrouper par eta
    eta_dict = {}
    for eta, pol in results:
        if eta not in eta_dict:
            eta_dict[eta] = []
        eta_dict[eta].append(pol)
    
    # Calculer moyenne et écart-type
    Etas = sorted(eta_dict.keys())
    Pol_mean = [np.mean(eta_dict[eta]) for eta in Etas]
    Pol_std = [np.std(eta_dict[eta]) for eta in Etas]

    return list(Etas), list(Pol_mean), list(Pol_std)

In [ ]:
# Paramètres
L = 5
v = 0.03
rho = 5
eta_c = 4

# Analyse
Etas, Pol, Pol_std = analyse_transition_distance(rho, L, v,3)

# Conversion en tableaux NumPy
Etas = np.array(Etas)
Pol = np.array(Pol)
Pol_std = np.array(Pol_std)

# Variables log-log
x = (eta_c - Etas) / eta_c
y = Pol

# Filtrage valeurs valides
mask = (x > 0) & (y > 0)
x = x[mask]
y = y[mask]

# Fit log-log
logx = np.log(x)
logy = np.log(y)

slope, intercept = np.polyfit(logx, logy, 1)

# Courbe de fit
x_fit = np.linspace(x.min(), x.max(), 200)
y_fit = np.exp(intercept) * x_fit**slope

# Tracé
plt.figure()
plt.loglog(x, y, 'o', label='Données')
plt.loglog(x_fit, y_fit, '-', 
        label=rf'Fit : $P \sim x^{{{slope:.2f}}}$')

plt.ylim(bottom=0.1)
plt.xlabel(r'$(\eta_c - \eta)/\eta_c$')
plt.ylabel(r'$\mathrm{Pol}$')
plt.grid(True, which='both')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Exposant (pente log-log) = {slope:.4f}")